# Analysis of Orbits in Fourier Space

Here we simulate Dan's Machine Learning system with rebound and then do a basic Fourier analysis. 

In [1]:
import rebound
import numpy as np
import pandas as pd
import numpy as np
import time

Here we set up the outer solar system with the possibility to inflate the masses. Such a system is known to go unstable on short timescales (e.g. Chambers, 1999). The simulation stops once there is a close encounter. 

In [7]:
def problem(a_outer,output_N,tmax,seed):
    sim = rebound.Simulation()
    sim.integrator = "whfast"
    sim.usleep = -1
    sim.dt = 1e-4*2*np.pi
    times = np.linspace(0,tmax*2*np.pi,output_N)   #In G=1 units, planet at 1AU takes 2pi years to orbit once.
    
    #parameters
    m_planet = 5*3e-6    #5-earth masses
    a_planet = 1
    rhill = a_planet*(m_planet/3.)**(1./3.)
    e_planet = 0.02
    inc = 0
    
    np.random.seed(seed)
    
    #add massive bodies
    sim.add(m=1.)
    sim.add(a=a_planet-52.63*rhill,m=m_planet,e=0,inc=0,f=np.random.random()*np.pi*2)
    sim.add(a=a_planet,m=m_planet,e=e_planet,inc=0,f=np.random.random()*np.pi*2)
    sim.add(a=a_outer,m=m_planet,e=0,inc=0,f=np.random.random()*np.pi*2)
    sim.N_active = sim.N
    
    #add test body
    #sim.add(a=a_outer,m=0,e=0,inc=0,f=np.random.random()*np.pi*2)
    
    sim.move_to_com()
    E0 = sim.calculate_energy()
    sim.init_megno()
    
    #condition to stop the simulation
    sim.exit_min_distance = a_planet*(m_planet/3.)**(1./3.)
    
    #output
    output_name = "Fourier/data/DTtestp_t"+str('{0:.1e}'.format(tmax))+"_aouter"+str(a_outer)+"_sd"+str(seed)+".csv"
    output_info = "Fourier/data/DTtestp_t"+str('{0:.1e}'.format(tmax))+"_aouter"+str(a_outer)+"_sd"+str(seed)+"_info.txt"
    f = open(output_info,"w")
    f.write("System Info: dt=%e, tmax=%d, N_output=%d\n"%(sim.dt,tmax,output_N))
    for i,p in enumerate(sim.particles[1:]):
        f.write("Planet %d: a=%f,m=%e,e=%f,inc=%f,f=%f\n"%(i+1,p.a,p.m,p.e,p.inc,p.f))
    f.close()
    
    #integrate
    start_time = time.time()
    print 'starting simulation for '+output_name
    with open(output_name, 'w') as f:
        try:
            for t in times:
                sim.integrate(t)
                f.write(str(sim.t/(2*np.pi)))
                f.write(','+str(sim.calculate_lyapunov()))
                outs = []
                for p in sim.particles[1:sim.N - sim.N_var]:
                    outs += [p.a, p.P, p.e, p.pomega, p.inc, p.Omega, p.f]
                for out in outs:
                    f.write(',{0}'.format(out))
                f.write('\n')
            f.close()
            o = open(output_info,"a")
            o.write('\nSimulation complete. Final energy error is: '+str(abs((E0 - sim.calculate_energy())/E0))+".")
            o.write('\nTotal Elapsed time is: '+str(time.time() - start_time)+' seconds.')
            o.close()
            print 'Simulation '+output_name+' complete. Final energy error is:', abs((E0 - sim.calculate_energy())/E0)
        except rebound.Encounter as error:
            f.write(str(sim.t/(2*np.pi)))
            f.write(','+str(sim.calculate_lyapunov()))
            outs = []
            for p in sim.particles[1:sim.N - sim.N_var]:
                outs += [p.a, p.P, p.e, p.pomega, p.inc, p.Omega, p.f]
            outs += []
            for out in outs:
                f.write(',{0}'.format(out))
            f.write('\n')
            f.close()
            o = open(output_info,"a")
            o.write("\nClose encounter, simulation "+output_name+" stopping at t="+str(sim.t/(2*np.pi))+" years.")
            o.write('\nTotal Elapsed time is: '+str(time.time() - start_time)+' seconds.')
            o.close()
            print "Close encounter, simulation "+output_name+" stopping at t="+str(sim.t/(2*np.pi))+" years."

In [10]:
#sample test
problem(1.5,16,1000,16)

starting simulation for Fourier/data/DTtestp_t1.0e+03_aouter1.5_sd16.csv
Simulation Fourier/data/DTtestp_t1.0e+03_aouter1.5_sd16.csv complete. Final energy error is: 1.0


Choose the parameters you will search over.

In [3]:
n_runs = 14             #number of simulations you want to run
tmax = 3e5              #simulation time
n_outputs = tmax*4      #number of outputted data points, important for FFT

**Warning!** Executing the code below will take a long time!

In [ ]:
#Get inputs
aouter = np.concatenate((1.082*np.ones(n_runs/2),1.211*np.ones(n_runs/2)), axis=0)
params = zip(aouter,n_outputs*np.ones(n_runs),tmax*np.ones(n_runs),(1000*np.random.random(n_runs)).astype("int"))

#Run the problem
import multiprocessing as mp
def execute(pars):
    problem(pars[0],pars[1],pars[2],pars[3])

if __name__== '__main__':
    pool = mp.Pool(processes=n_runs)
    pool.map(execute, params)
    pool.close()
    pool.join()

starting simulation for Fourier/data/DTsys_t1.0e+06_HRF3.56_sd65.csv
starting simulation for Fourier/data/DTsys_t1.0e+06_HRF3.0_sd60.csv
starting simulation for Fourier/data/DTsys_t1.0e+06_HRF4.11_sd683.csv
starting simulation for Fourier/data/DTsys_t1.0e+06_HRF5.22_sd824.csv
starting simulation for Fourier/data/DTsys_t1.0e+06_HRF6.33_sd149.csv
starting simulation for Fourier/data/DTsys_t1.0e+06_HRF5.78_sd819.csv
starting simulation for Fourier/data/DTsys_t1.0e+06_HRF6.89_sd265.csv
starting simulation for Fourier/data/DTsys_t1.0e+06_HRF7.44_sd316.csv
starting simulation for Fourier/data/DTsys_t1.0e+06_HRF8.0_sd358.csv
starting simulation for Fourier/data/DTsys_t1.0e+06_HRF4.67_sd339.csv


/Users/silburt/phd/rebound/rebound/simulation.py:301: RuntimeWarning: Configuring box automatically for vizualization based on particle positions.
  warnings.warn(msg[1:], RuntimeWarning)
/Users/silburt/phd/rebound/rebound/simulation.py:301: RuntimeWarning: Configuring box automatically for vizualization based on particle positions.
  warnings.warn(msg[1:], RuntimeWarning)
/Users/silburt/phd/rebound/rebound/simulation.py:301: RuntimeWarning: Configuring box automatically for vizualization based on particle positions.
  warnings.warn(msg[1:], RuntimeWarning)
/Users/silburt/phd/rebound/rebound/simulation.py:301: RuntimeWarning: Configuring box automatically for vizualization based on particle positions.
  warnings.warn(msg[1:], RuntimeWarning)
/Users/silburt/phd/rebound/rebound/simulation.py:301: RuntimeWarning: Configuring box automatically for vizualization based on particle positions.
  warnings.warn(msg[1:], RuntimeWarning)
/Users/silburt/phd/rebound/rebound/simulation.py:301: Runtim

Close encounter, simulation Fourier/data/DTsys_t1.0e+06_HRF3.0_sd60.csv stopping at t=2.0165 years.
Close encounter, simulation Fourier/data/DTsys_t1.0e+06_HRF3.56_sd65.csv stopping at t=100.882800003 years.
Close encounter, simulation Fourier/data/DTsys_t1.0e+06_HRF4.11_sd683.csv stopping at t=164.807800006 years.


Process PoolWorker-2:
Process PoolWorker-3:
Process PoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/silburt/anaconda2/envs/ml/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/silburt/anaconda2/envs/ml/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/silburt/anaconda2/envs/ml/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
  File "/Users/silburt/anaconda2/envs/ml/lib/python2.7/multiprocessing/process.py", line 114, in run
    self.run()
  File "/Users/silburt/anaconda2/envs/ml/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
    self._target(*self._args, **self._kwargs)
  File "/Users/silburt/anaconda2/envs/ml/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/Users/silburt/anaconda2/envs/ml/lib/python2.7/multiproce

Now go to Fourier_Analyze notebook to analyze the results.